In [0]:
from config import DeployConfig
import requests
import json

In [0]:
dbutils.widgets.text("config_path", "./config/env_variables.yml")
config_path = dbutils.widgets.get("config_path")
cfg = DeployConfig.from_yaml(config_path)

In [0]:
image_table = getattr(cfg, f"image_table")
endpoint_name = getattr(cfg, f"endpoint_name")

In [0]:
endpoint_name

# OPEN API SCHEMA OF ENDPOINT

In [0]:
endpoint_url = f"https://e2-demo-field-eng.cloud.databricks.com/api/2.0/serving-endpoints/{endpoint_name}/openapi"


notebook_token = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()

headers = {
    "Content-Type": "application/json",
    "Authorization": f'Bearer {notebook_token}'
}

response = requests.get(endpoint_url, headers=headers).json()

display(response)

# BATCH INFERENCE TO COMPUTE EMBEDDINGS

Apply ai_query to compute embeddings of our images before syncing to Vector Search Index

In [0]:
%sql
select schema_of_json("{'predictions': [{'model_input': [-0.3264505863189697, 0.8023356199264526]}]}")

In [0]:
%sql
select schema_of_json("[{'model_input': [-0.3264505863189697, 0.8023356199264526]}]")

In [0]:
%sql
select * from ml_shovakeemian.feip.cat_images limit 10;

In [0]:
spark.sql(
  f"""
  CREATE OR REPLACE TABLE {image_table.path}_embeddings
  AS 
  SELECT 
    id,
    path,
    ai_query('{endpoint_name}', 
      request =>  named_struct(
        "model_input", model_input
      ),
      returnType => 'ARRAY<STRUCT<model_input: ARRAY<DOUBLE>>>'
      )[0].model_input as image_embeddings
FROM {image_table.path}
"""
)
